<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/chatbot_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
OptiBot Chatbot Controller

This module provides the main backend logic for the OptiLine virtual assistant chatbot ("OptiBot").
The chatbot can answer user questions about the OptiLine system, leveraging both generative AI (Gemini) and live data from Firebase when needed.

Functions:
    - ask_optibot(question: str) -> str
        Main entry point. Receives a user question (string) and returns a single English answer as a string.
        Answers can include both real-time data from the system and general knowledge.

Dependencies:
    - google-generativeai (for Gemini API)
    - importnb (for loading other project notebooks as modules)
    - Firebase helper modules (existing: FireBase, user_controller, etc.)
    - Colab secrets (for securely storing Gemini API key)

Note:
    This module is backend-only; it does not include any frontend or UI logic.
    All code comments and docstrings are in English for documentation purposes.
"""

In [ ]:
# ─── Imports ───

import google.generativeai as genai
from google.colab import userdata
import time
# Import project modules using importnb
from importnb import Notebook

# Load Firebase and helper modules as notebooks
with Notebook():
    import FireBase as fb           # Firebase data access functions
    import user_controller as uc    # User management and leaderboard functions
    import mqqt_sim_indoor as indoor    # Indoor sensor simulator
    import mqqt_sim_outdoor as outdoor  # Outdoor sensor simulator

# ─── Constants ───

# --- OptiBot Official Documentation Context (Knowledge Base) ---
OPTI_BOT_CONTEXT = """
You are **OptiBot**, the official assistant for the OptiLine Dashboard (CIM & Robotics Lab, Braude College).

╭──────────────────────────────────────────────╮
│  ANSWER POLICY                               │
│  ───────────                                 │
│  • If the user asks about OptiLine—the lab,  │
│    dashboard sections, sensors, faults,      │
│    leaderboard, or points—answer **only**    │
│    from the documentation below.             │
│  • If the user asks a general question       │
│    (current time/date, who you are, basic    │
│    small-talk/help, weather), answer freely  │
│    using general knowledge.                  │
│  • Never invent OptiLine-specific facts.     │
╰──────────────────────────────────────────────╯


────────────────────────────────────────
# About the CIM & Robotics Laboratory
────────────────────────────────────────
The CIM & Robotics Laboratory at Braude College of Engineering, established in 1997, serves as an advanced learning environment for students in Industrial and Mechanical Engineering.
Located in **Room D106**, the lab includes semi-industrial CIM (Computer Integrated Manufacturing) and FMS (Flexible Manufacturing System) platforms.

**Hands-on systems:**
- Robotic arms & assembly cells
- CNC machining & simulation
- CAD/CAM tools
- Vision-based quality control
- Programmable-logic controllers (PLCs)
- Automated storage & retrieval system (AS/RS)

Students engage in real-world manufacturing scenarios that bridge theoretical knowledge with industry-relevant skills.


────────────────────────────────────────
# About the Application – OptiLine Dashboard
────────────────────────────────────────
A cloud-based interface for engineers working with the autonomous production line in the CIM lab. It provides:
- Real-time sensor monitoring (indoor & outdoor)
- Historical data analysis with trend visualisation
- Search over MQTT-related technical content
- Fault Simulator with daily challenges
- Gamified scoring & leaderboard
- User directory and personalised view

Accessible securely from any device; supports **live** and **simulated** data modes.


────────────────────────────────────────
# Available Sensors
────────────────────────────────────────
| Sensor | Description                              | Unit | Location            |
|--------|------------------------------------------|------|---------------------|
| Temperature | Ambient temperature                | °C   | Indoor & Outdoor    |
| Humidity    | Relative air humidity              | %    | Indoor & Outdoor    |
| Pressure    | Atmospheric pressure               | hPa  | Indoor              |
| Dlight      | Light intensity (Illuminance)      | Lux  | Outdoor             |
| Distance    | Ultrasonic movement detection      | mm   | Indoor              |

*All sensors stream data in real time via MQTT.*


────────────────────────────────────────
# Scoring & Leaderboard
────────────────────────────────────────
- **Earn points** by completing Fault Simulator challenges.
- **Difficulty** ↑ = points ↑.
- **Faster** completion improves ranking.
- **Full completion required** – no partial credit.
- Points accumulate to determine your leaderboard rank (updates after each completed challenge).


────────────────────────────────────────
# FAQ (Official)
────────────────────────────────────────
**Q:** Can I skip steps in the Fault Simulator and still get points?
**A:** No. You must complete all required steps before submitting. Partial credit is not available.

**Q:** Can I view or repeat previous faults?
**A:** Not at this stage. Handled faults cannot be revisited.

**Q:** Are the sensors live or simulated?
**A:** Both modes are supported. In simulation mode, realistic data is streamed from a pre-loaded scenario.

**Q:** How often is the leaderboard updated?
**A:** After each completed challenge.


────────────────────────────────────────
# Main Dashboard Sections
────────────────────────────────────────
1. **Live Sensors Dashboard** – monitor real-time data.
2. **Statistics Panel** – interactively explore historical trends.
3. **MQTT Search Engine** – search technical content.
4. **Fault Simulator** – practise troubleshooting.
5. **User Directory** – see authorised contacts.
6. **Leaderboard** – track cumulative performance points.
"""




# The Gemini API key is securely retrieved from Colab secrets
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")


In [ ]:
# -------------------  helper --------------------
OPTI_KEYWORDS = [
    "optiline", "cim", "robotics lab", "sensor", "temperature", "humidity",
    "pressure", "dlight", "distance", "fault", "simulator",
    "leaderboard", "points", "dashboard", "statistics panel",
    "user directory", "mqtt"
]

def _is_optiline(q: str) -> bool:
    """Return True if the question contains any OptiLine-related keyword."""
    return any(kw in q for kw in OPTI_KEYWORDS)

# -------------------  main ---------------------
def ask_optibot(message: str, chat_history: list | None) -> tuple[str, list]:
    """
    Unified OptiBot handler

    Order of operations
    -------------------
    1. General whitelist (time / date / who-are-you / help / weather)
    2. Live leaderboard (top-3)
    3. Live sensor value (indoor / outdoor)
    4. FAQ from official KB (exact or fuzzy match)
    5. Free Gemini answer (only if the question is NOT OptiLine related)
    6. Refusal
    """
    if chat_history is None:
        chat_history = []

    chat_history.append({"role": "user", "content": message})
    q = message.lower().strip()

    # 1 ─ General whitelist
    match = _match_general(q)
    if match:
        if match == "time":
            ans = f"The current time is {time.strftime('%H:%M')} (Israel)."
        elif match == "date":
            ans = time.strftime("Today is %A, %B %d, %Y.")
        elif match == "who":
            ans = "I am OptiBot, the official assistant for the OptiLine Dashboard."
        elif match == "help":
            ans = (
                "I can provide live sensor values, leaderboard status, Fault-Simulator information, "
                "and answers from the official user guide. Feel free to ask."
            )
        else:  # weather
            ans = "I do not have real-time weather data; please check a weather service."
        chat_history.append({"role": "assistant", "content": ans})
        return "", chat_history

    # 2 ─ Live leaderboard (top-3)
    if "leaderboard" in q and any(w in q for w in ("top", "rank", "score")):
        top, _ = uc.get_leaderboard("", top_n=3)
        ans = (
            "Current top-3:\n" +
            "\n".join(f"{r}. {n} – {s} pts" for r, n, s in top)
        ) if top else "Leaderboard data is not available."
        chat_history.append({"role": "assistant", "content": ans})
        return "", chat_history

    # 3 ─ Live sensor value
    for key, (field, unit, src_in, src_out) in SENSOR_MAP.items():
        if key in q:
            src = src_in if "indoor" in q else src_out if "outdoor" in q else src_out
            if not src:
                ans = "That sensor is not available at the requested location."
            else:
                try:
                    val = next(src.get_live_data_stream(mode="simulation"))[field]
                    lo, hi = NORMAL_RANGE[field]
                    status = "Normal" if lo <= val <= hi else ("High" if val > hi else "Low")
                    loc    = "indoor" if src is src_in else "outdoor"
                    ans = f"Current {loc} {field.lower()}: {val} {unit} ({status})."
                except Exception as exc:
                    ans = f"Could not retrieve sensor data ({exc})."
            chat_history.append({"role": "assistant", "content": ans})
            return "", chat_history

    # 4 ─ FAQ from KB (exact / fuzzy)
    if any(kw in q for kw in key_map):               # exact
        topic = next(t for kw, t in key_map.items() if kw in q)
        chat_history.append({"role": "assistant", "content": FAQ_ANSWERS[topic]})
        return "", chat_history

    fuzzy = difflib.get_close_matches(q, all_keys, n=1, cutoff=0.7)
    if fuzzy:
        chat_history.append({"role": "assistant",
                             "content": FAQ_ANSWERS[key_map[fuzzy[0]]]})
        return "", chat_history

    # 5 ─ Free Gemini answer for non-OptiLine topics
    if not _is_optiline(q):
        try:
            if GEMINI_API_KEY:
                genai.configure(api_key=GEMINI_API_KEY)
                model   = genai.GenerativeModel("gemini-1.5-flash")
                reply   = model.generate_content(message)
                ans = reply.text.strip() if hasattr(reply, "text") else str(reply)
            else:
                ans = "The language-model API key is not configured."
        except Exception as exc:
            ans = f"An error occurred: {exc}"
        chat_history.append({"role": "assistant", "content": ans})
        return "", chat_history

    # 6 ─ Refusal (OptiLine topic but no match)
    refusal = (
        "I can answer only according to the official OptiLine documentation. "
        "Please ask about system features, usage, scoring, or sensors."
    )
    chat_history.append({"role": "assistant", "content": refusal})
    return "", chat_history
